# Notebook for evaluating various test function errors ONLY AFTER the coresets are already generated

In [ ]:
import numpy as np
import numpy.random as npr
import numpy.linalg as npl
from scipy.spatial.distance import pdist

from argparse import ArgumentParser
import pickle as pkl
import pathlib
import os
import os.path

# import kernel thinning
from goodpoints import kt # kt.thin is the main thinning function; kt.split and kt.swap are other important functions
from goodpoints.util import isnotebook # Check whether this file is being executed as a script or as a notebook
from goodpoints.util import fprint  # for printing while flushing buffer
from goodpoints.tictoc import tic, toc # for timing blocks of code


# utils for generating samples, evaluating kernels, and mmds
from util_sample import sample, compute_mcmc_params_p, compute_diag_mog_params, sample_string, compute_params_p
from util_k_mmd import kernel_eval, p_kernel, ppn_kernel, pp_kernel, pnpn_kernel, squared_mmd, get_combined_results_filename, compute_params_k
from util_parse import init_parser, convert_arg_flags

# for partial functions, to use kernel_eval for kernel
from functools import partial

# set things a bit when running the notebook
if isnotebook():
    # Autoreload packages that are modified
    %load_ext autoreload
    %autoreload 2
    %matplotlib inline
    %load_ext line_profiler
    # https://jakevdp.github.io/PythonDataScienceHandbook/01.07-timing-and-profiling.html

## 1. Function for loading input and coresets

In [ ]:
def load_input_and_coreset(m, params_p, params_k_split, params_k_swap, rep_id, thin_str="", delta=0.5, 
                      sample_seed=1234567, thin_seed=9876543, results_dir="results_new", verbose=False):
    """Return exisiting KT coresets by loading from disk, and the associated MC points used for finding the coresets
   along with ST coresets
   Gives error if the coreset does not exist
    Args:
      m: Number of halving rounds (number of sample points n = 2^{2m})
      params_p: Dictionary of distribution parameters recognized by sample()
      params_k_split: Dictionary of kernel parameters recognized by kernel() # used for kt split
      params_k_swap: Dictionary of kernel parameters recognized by kernel() # used for kt swap; and computing mmd
      rep_id: A single rep id for which coreset to be returned
      thin_str: (Optional), str to be appended to filenames when loading coresets other than KT and iid/ST, e.g., kt.split + rand
      delta: If c is None, delta/(4^m) is the failure probability for
        adaptive threshold sequence
      sample_seed: (Optional) random seed is set to sample_seed + rep
        prior to generating input sample for replication rep
      thin_seed: (Optional) random seed is set to thin_seed + rep
        prior to running thinning for replication rep
      results_dir: (Optional) Directory in which results is to be loaded from
      verbose: (Optional) If True, print intermediate updates
    """
    
    d = params_p["d"]
    assert(d == params_k_split["d"])
    assert(d == params_k_swap["d"])
    sample_str = sample_string(params_p, sample_seed)
    split_kernel_str = "{}_var{:.3f}_seed{}".format(params_k_split["name"], params_k_split["var"], thin_seed)
    swap_kernel_str =  "{}_var{:.3f}".format(params_k_swap["name"], params_k_swap["var"])
    thresh_str = f"delta{delta}"
    file_template = os.path.join(results_dir, f"kt{thin_str}-coresets-{sample_str}-split{split_kernel_str}-swap{swap_kernel_str}-d{d}-m{m}-{thresh_str}-rep{{}}.pkl")
    
    filename = file_template.format(rep_id)
    n = int(2**(2*m))
    ncoreset = int(2**m)
    X = sample(n, params_p, seed=sample_seed+rep_id)
    if os.path.exists(filename):
        with open(filename, 'rb') as file:
            if verbose:
                print(f"Loading KT coreset indices from {filename}")
            coresets = pkl.load(file)
    else:
        raise ValueError(f"File {filename} not found")

    if verbose:
        print(f"Returning all {n} input MC points and {ncoreset} KT points")
        
    return(X, coresets[:ncoreset])
    

## 2. Function for evaluating integration errors

In [ ]:
def evaluate_fun_approx_quality(fun_str, ms, params_p, params_k_split, params_k_swap, rep_ids,
        thin_str="",  delta=0.5,  sample_seed=1234567, thin_seed=9876543,
        compute_fun_diff = True, rerun=False, results_dir="results_new", return_val=False):
    """Returns |Pinf-Pout f|, |Pf - Poutf| for KT/KTrt/KT+ and ST
    
    Args:
    fun_str: the test function to be evaluated must be in the list
             {k0, x, x^2, pk, kmean, x1x2, cov, l1_x, linf_x, cif, gfun, cos, cosg, kernel}
             - some functions might have constraints on what settings allowed
             - add another function by adding an "if block" in this code
      ms: range of output coreset sizes (2^m for m in ms)
      params_p: Dictionary of distribution parameters recognized by sample()
      params_k_split: Dictionary of kernel parameters recognized by kernel_eval()
      params_k_swap: Dictionary of kernel parameters recognized by kernel_eval()
      rep_ids: Which replication numbers of experiment to run; the replication
        number determines the seeds set for reproducibility
      thin_str: (Optional), str to be appended to filenames when loading coresets other than KT/KT power and iid/ST, e.g., "-plus" for KT+
      delta: delta/(4^m) is the failure probability for
        adaptive threshold sequence;
      sample_seed: (Optional) random seed is set to sample_seed + rep
        prior to generating input sample for replication rep
      thin_seed: (Optional) random seed is set to thin_seed + rep
        prior to running thinning for replication rep
      compute_fun_diff: (Optional)
      rerun: (Optional) If False and results have been previously saved to
        disk, load results from disk instead of recomputing the errors
      results_dir: (Optional) Directory in which results should be saved
      return_val:(Optional) Whether to return the |Pinf-Pout f|, |Pf - Poutf| for KT, and ST (4 quantities in total) OR not
    """
    # Create results directory if necessary
    pathlib.Path(results_dir).mkdir(parents=True, exist_ok=True)
    d = params_p["d"]
    assert(d==params_k_split["d"])
    assert(d==params_k_swap["d"])
    
    # create split and swap kernel functions from the parameters
    split_kernel = partial(kernel_eval, params_k=params_k_split)
    swap_kernel = partial(kernel_eval, params_k=params_k_swap)
    
    # Construct results filename template with placeholder for rep value
    sample_str = sample_string(params_p, sample_seed)
    split_kernel_str = "{}_var{:.3f}_seed{}".format(params_k_split["name"], params_k_split["var"], thin_seed)
    swap_kernel_str =  "{}_var{:.3f}".format(params_k_swap["name"], params_k_swap["var"])
    thresh_str = f"delta{delta}"
    
    orig_fun_str = fun_str # for easeness, as we change fun_str to use previous results
    if fun_str == 'k0': fun_str = "" # changed to remain consistent with previous computations

    ########### DEFINE FUNCTIONS AND COMPUTE Pf WHENEVER COMPUTABLE ##########
    # If Pf is not available, then we later set Pf = Pinf (which is always COMPUTABLE)
    
    if fun_str == "": # f(x) = k(0, x)
        yloc = np.zeros((1, d))
        fun = partial(swap_kernel, y=yloc)
        p_fun = p_kernel(yloc, params_k=params_k_swap, params_p=params_p)[0] # fun is fixed to be k(yloc, .)
        
    if fun_str == 'pk':
        fun = partial(p_kernel, params_k=params_k_swap, params_p=params_p)
        p_fun = pp_kernel(params_k_swap, params_p)
        
    if fun_str == 'kmean': # f(x)=Pk(x), enabled only for MCMC experiments where P is fixed to Phat/Pnmax
        assert("Pnmax" in params_p)
        yloc = params_p["Pnmax"].mean(0).reshape(1, -1) # mean over samples
        fun = partial(swap_kernel, y=yloc)
        p_fun = p_kernel(yloc, params_k=params_k_swap, params_p=params_p)[0] # fun is fixed to be k(yloc, .)
    
    if fun_str == 'x': # first coordinate
        def fun(x): return(x[:,0])
        if params_p["name"] == "gauss":
            p_fun = 0.
        if "Pnmax" in params_p:
            p_fun = np.mean(fun(params_p["Pnmax"]))
        if params_p["name"] == "diag_mog" and (len(params_p["weights"])==4 or len(params_p["weights"])==8):
            p_fun = 0.
            
    if fun_str == 'x1x2': # product of first two coordinates
        def fun(x): return(x[:,0]*x[:,1])
        if params_p["name"] == "gauss":
            p_fun = 0.
        if "Pnmax" in params_p:
            p_fun = np.mean(fun(params_p["Pnmax"]))
        if params_p["name"] == "diag_mog" and (len(params_p["weights"])==4 or len(params_p["weights"])==8):
            p_fun = 0.
            
    if fun_str == 'cov': # Covariance function E[(X-mu)(X-mu)^T]
        assert(params_p["name"]=="gauss")
        def fun(X): return(X.T.dot(X)/X.shape[0]-np.outer(X.mean(0),X.mean(0)))
        if params_p["name"] == "gauss":
            p_fun = params_p["var"]*np.eye(d)
    
    if fun_str == 'l1_x' or fun_str == 'linf_x': # want to compute |P X-Pout X|_1 and |P X-Pout X|_inf, so here we compute PX
        def fun(x): return(x)
        if params_p["name"] == "gauss":
            p_fun = np.zeros(d)
        if "Pnmax" in params_p:
            p_fun = np.mean(fun(params_p["Pnmax"]), 0)
        if params_p["name"] == "diag_mog" and (len(params_p["weights"])==4 or len(params_p["weights"])==8):
            p_fun = np.zeros(d)
     
    if fun_str == 'x^2': # first coordinate squared
        def fun(x): return(x[:, 0]**2)
        if params_p["name"] == "gauss":
            p_fun = params_p["var"]
        if "Pnmax" in params_p:
            p_fun = np.mean(fun(params_p["Pnmax"]), 0)
        if params_p["name"] == "diag_mog" and (len(params_p["weights"])==4 or len(params_p["weights"])==8):
            p_fun = params_p["mean_sqdist"]/4.
        
    # for these functions Pf is not directly available; and we only compute Pinf (And set Pf = Pinf)
    if fun_str == 'cif':  # https://www.sfu.ca/~ssurjano/cont.html
        def fun(x): 
            # function from here
            d = x.shape[1]
            u = npr.default_rng(0).uniform(size=(1, d))
            a = 1. / d * np.ones(d)
            return(np.exp(np.sum(-np.abs(x-u.reshape(1, -1)) * a, axis=1 ) ))
        
    if fun_str == "gfun": # https://www.sfu.ca/~ssurjano/gfunc.html
        def fun(x):
            # function from here
            d = x.shape[1]
            a = 0.5*np.arange(1, d+1)  - 1
            return(np.prod((np.abs(4*x - 2) + a.reshape(1, -1) ) / (1+a), axis=1))
    
    if fun_str == "cos": # cosine function; https://www.sfu.ca/~ssurjano/oscil.html
        def fun(x):
            d = x.shape[1] 
            u = npr.default_rng(0).uniform()
            return(np.cos(2*np.pi*u+ 5./d * np.sum(x , axis=1 ) ))
    
    if fun_str == "cosg": # cosine * gaussian function; p
        def fun(x):
            d = x.shape[1] 
            u = npr.default_rng(0).uniform()
            return(np.exp(-5./d*np.sum(x**2, axis=1)) * np.cos(2*np.pi*u+ 5./d * np.sum(x , axis=1 ) ))
        
    if fun_str == "kernel": # f(x) = k(X', x) for X' ~ P
        if params_p["name"]  != "gauss" and "mog" not in params_p["name"]:
            u = npr.default_rng(100).choice(len(params_p["Pnmax"]))
            u = 2 * params_p["Pnmax"][u] - np.mean(params_p["Pnmax"], 0)
        else:
            # generate a randomvariable
            u = sample(1, params_p,  npr.default_rng(0)) # 2 * np.sqrt(params_p["var"]) * npr.default_rng(0).standard_normal(size=(1, params_p["d"]))
        def fun(x):
            d = x.shape[1] 
            return(kernel_eval(x, u, params_k=params_k_swap))
        if params_p["name"] == "gauss" and params_k["name"] == "gauss":
            p_fun = p_kernel(y=u, params_k=params_k_swap, params_p=params_p)            
       
    ########### COMPUTE Pinf and Poutf  ##########
    
    # initialize matrices of entries       
    fun_diff_p = np.zeros((len(ms), len(rep_ids))) # for Pf - Pout f
    fun_diff_p_sin = np.zeros((len(ms), len(rep_ids))) # Pinf - Pout f
    fun_diff_p_st = np.zeros((len(ms), len(rep_ids))) #  Pf - Pout f for standard thinning
    fun_diff_p_sin_st = np.zeros((len(ms), len(rep_ids))) # Pinf - Pout f for standard thinning
    
    fprint(f"Evaluating coresets for function {orig_fun_str} for setting \
           {get_combined_results_filename('', ms, params_p, params_k_split=params_k_split, params_k_swap=params_k_swap, rep_ids=rep_ids, delta=delta)}.....")
    generic_prefixes = [f"-combinedfundiff{fun_str}-", f"-sin-combinedfundiff{fun_str}-"]
    
    compute_st = False
    compute_kt = False
    # check if things are already stored then don't compute the respective items
    if not rerun:
        prefixes = ["mc" + prefix for prefix in generic_prefixes]
        for prefix in prefixes:
            filename = get_combined_results_filename(prefix, ms, params_p, params_k_split=params_k_split, params_k_swap=params_k_swap, rep_ids=rep_ids, delta=delta)
            if not os.path.exists(filename):
                compute_st = True

        prefixes = [f"kt{thin_str}" + prefix for prefix in generic_prefixes]
        for prefix in prefixes:
            filename = get_combined_results_filename(prefix, ms, params_p, params_k_split=params_k_split, params_k_swap=params_k_swap, rep_ids=rep_ids, delta=delta)
            if not os.path.exists(filename):
                compute_kt = True
    else:
        compute_st = True
        compute_kt = True
    
    if compute_st or compute_kt:
        for m in ms:
            # print(m)
            for r_i, rep in enumerate(rep_ids):
                # load coresets
                Xin, kt_coresets = load_input_and_coreset(m, params_p, params_k_split, params_k_swap, rep_id=rep, thin_str=thin_str, delta=delta, 
                              sample_seed=sample_seed, thin_seed=thin_seed, results_dir=results_dir, verbose=False)
                
                # compute Pinf for various functions
                if fun_str == 'cov':
                    pin_fun = fun(Xin)
                elif fun_str not in ['cif', 'gfun', 'cos' ,'cosg', 'kernel']:
                    pin_fun = np.mean(fun(Xin), 0) if not params_p["saved_samples"] else p_fun # to save time, ignore pk setting with pin for mcmc cases
                else:
                    pin_fun = np.mean(fun(Xin), 0)
                    
                if 'p_fun' not in locals(): p_fun = pin_fun
                
                # compute Pout f for KT
                if compute_kt:
                    if fun_str == 'cov':
                        pout_fun_kt = fun(Xin[kt_coresets])
                    else:
                        pout_fun_kt = np.mean(fun(Xin[kt_coresets]), 0)
                   
                    if fun_str == 'l1_x' or fun_str == 'cov':
                        multiply_factor = 1. if fun_str == 'l1_x' else 1./d**2 # normalize by d^2 for covariance
                        fun_diff_p[m, r_i] = multiply_factor * np.sum(np.abs(p_fun-pout_fun_kt))
                        fun_diff_p_sin[m, r_i] =  multiply_factor * np.sum(np.abs(pin_fun-pout_fun_kt))
                    elif fun_str == 'linf_x':
                        fun_diff_p[m, r_i] = max(np.abs(p_fun-pout_fun_kt))
                        fun_diff_p_sin[m, r_i] = max(np.abs(pin_fun-pout_fun_kt))
                    else:
                        fun_diff_p[m, r_i] = np.abs(p_fun-pout_fun_kt)
                        fun_diff_p_sin[m, r_i] = np.abs(pin_fun-pout_fun_kt)
                   
                # compute Poutf for ST
                if compute_st:
                    step = int(2**m)
                    if fun_str == 'cov':
                        pout_fun_st = fun(Xin[step-1:int(2**(2*m)):step])
                    else:
                        pout_fun_st = np.mean(fun(Xin[step-1:int(2**(2*m)):step]))
                        
                    if fun_str == 'l1_x' or fun_str == 'cov':
                        multiply_factor = 1. if fun_str == 'l1_x' else 1./d**2
                        fun_diff_p_st[m, r_i] =  multiply_factor * np.sum(np.abs(p_fun-pout_fun_st))
                        fun_diff_p_sin_st[m, r_i] = multiply_factor * np.sum(np.abs(pin_fun-pout_fun_st))
                    elif fun_str == 'linf_x':
                        fun_diff_p_st[m, r_i] = max(np.abs(p_fun-pout_fun_st))
                        fun_diff_p_sin_st[m, r_i] = max(np.abs(pin_fun-pout_fun_st))
                    else:
                        fun_diff_p_st[m, r_i] = np.abs(p_fun-pout_fun_st)
                        fun_diff_p_sin_st[m, r_i] = np.abs(pin_fun-pout_fun_st)
    
    ########### SAVE RESULTS ########## 
    # stanfard thinning results are saved with "mc prefix" 
    prefixes = ["mc" + prefix for prefix in generic_prefixes]
    if compute_st:      
        for prefix, data_array in zip(prefixes, [fun_diff_p_st, fun_diff_p_sin_st]):
            filename = get_combined_results_filename(prefix, ms, params_p, params_k_split=params_k_split, params_k_swap=params_k_swap, rep_ids=rep_ids, delta=delta)
            with open(filename, 'wb') as file:
                print(f"Saving {prefix} to {filename}")
                pkl.dump(data_array, file, protocol=pkl.HIGHEST_PROTOCOL)
    else:
        if return_val:
            prefix = prefixes[0]
            filename = get_combined_results_filename(prefix, ms, params_p, params_k_split=params_k_split, params_k_swap=params_k_swap, rep_ids=rep_ids, delta=delta)
            with open(filename, 'rb') as file:
                print(f"Loading {prefix} from {filename}")
                fun_diff_p_st = pkl.load(file)

            prefix = prefixes[1]
            filename = get_combined_results_filename(prefix, ms, params_p, params_k_split=params_k_split, params_k_swap=params_k_swap, rep_ids=rep_ids, delta=delta)
            with open(filename, 'rb') as file:
                print(f"Loading {prefix} from {filename}")
                fun_diff_p_sin_st = pkl.load(file)

    prefixes = [f"kt{thin_str}" + prefix for prefix in generic_prefixes]      
    if compute_kt:
        for prefix, data_array in zip(prefixes, [fun_diff_p, fun_diff_p_sin]):
            filename = get_combined_results_filename(prefix, ms, params_p, params_k_split=params_k_split, params_k_swap=params_k_swap, rep_ids=rep_ids, delta=delta)
            with open(filename, 'wb') as file:
                print(f"Saving {prefix} to {filename}")
                pkl.dump(data_array, file, protocol=pkl.HIGHEST_PROTOCOL)
    else:
        if return_val:
            prefix = prefixes[0]
            filename = get_combined_results_filename(prefix, ms, params_p, params_k_split=params_k_split, params_k_swap=params_k_swap, rep_ids=rep_ids, delta=delta)
            with open(filename, 'rb') as file:
                print(f"Loading {prefix} from {filename}")
                fun_diff_p = pkl.load(file)

            prefix = prefixes[1]
            filename = get_combined_results_filename(prefix, ms, params_p, params_k_split=params_k_split, params_k_swap=params_k_swap, rep_ids=rep_ids, delta=delta)
            with open(filename, 'rb') as file:
                print(f"Loading {prefix} from {filename}")
                fun_diff_p_sin = pkl.load(file)
            
    if return_val:
        return(fun_diff_p, fun_diff_p_sin, fun_diff_p_st, fun_diff_p_sin_st)


# Initialize argumennts

In [ ]:
# if isnotebook():
parser = init_parser()
args, opt = parser.parse_known_args()
args = convert_arg_flags(args)

# Gauss Experiments Results


In [ ]:
rerun = True # whether to rerun the integratione rror computations
args.d = 2 # d
args.kernel = "gauss" # kernel
args.P = "gauss" # target P setting; allowed values depend on the feasible arguments in compute_params_p; currently {gauss, mog, mcmc} 
args.computepower = True # whether to compute results for power KT (same as root KT when power = 0.5)
args.power = 0.5 # power of the kernel to be used for power KT and KT+
args.ktplus = False # whether to run results for KT+
args.targetkt = True # whether to run results for KT+
args.powerkt = True # whether to run results for KT+
fun_strs = ['kernel', 'x', 'cif'] # which functions to evaluate

In [ ]:
# for gauss P the only free parameter is dimension d; everything else is computed in compute_params_p/compute_params_k
ds = [2, 10, 20, 50, 100] 
for d in ds:
    args.d = d
    d, params_p, var_k =  compute_params_p(args)
    params_k, params_k_power = compute_params_k(args, var_k, args.computepower, args.power)
    
    if args.ktplus: # if running KT+, need to define the KT+ kernel called as params_k_combo
        assert(args.power is not None)
        params_k_combo = dict()
        params_k_combo["name"] = "combo_"  + params_k["name"] + f"_{args.power}"
        params_k_combo["k"] = params_k.copy()
        params_k_combo["kpower"] = params_k_power.copy()
        params_k_combo["var"] = params_k["var"]
        params_k_combo["d"] = args.d
        
    
    params_k_split_list = []
    thin_str_list = []
    if args.targetkt:
        params_k_split_list.append(params_k)
        thin_str_list.append("")
    if args.powerkt:
        params_k_split_list.append(params_k_power)
        thin_str_list.append("")
    if args.ktplus:
        params_k_split_list.append(params_k_combo)
        thin_str_list.append("-plus")

    for fun_str in fun_strs:
        for params_k_split, thin_str in zip(params_k_split_list, thin_str_list):
            fun_diff_p, fun_diff_p_sin, fun_diff_p_st, fun_diff_p_sin_st = evaluate_fun_approx_quality(fun_str=fun_str,
                ms=range(7+1), params_p=params_p, params_k_split=params_k_split, params_k_swap=params_k, rep_ids=range(10),
                                            thin_str=thin_str, 
                             delta=0.5,
                              sample_seed=1234567, thin_seed=9876543,
                              rerun=rerun, results_dir="results_new",return_val = True)


## MCMC results

In [ ]:
all_mcmc_filenames = np.array(['Hinch_P_seed_1_temp_1_scaled', 'Hinch_P_seed_2_temp_1_scaled', 
                               'Hinch_TP_seed_1_temp_8_scaled', 'Hinch_TP_seed_2_temp_8_scaled', 
                                'Goodwin_RW_float_step', 'Goodwin_ADA-RW_float_step', 
                               'Goodwin_MALA_float_step',  'Goodwin_PRECOND-MALA_float_step',  
                               'Lotka_RW_float_step',  'Lotka_ADA-RW_float_step', 
                               'Lotka_MALA_float_step', 'Lotka_PRECOND-MALA_float_step'])

## Lotka and Goodwin

In [ ]:
rerun = True
args.d = int(4)
args.kernel, args.power = "laplace", 0.81 # we used different power for different kernels
# args.kernel, args.power = "imq", 0.5
# args.kernel, args.power = "gauss", 0.5
args.P = "mcmc" # target P setting; allowed values depend on the feasible arguments in compute_params_p; currently {gauss, mog, mcmc} 
args.computepower = True  # whether to compute results for power KT (same as root KT when power = 0.5)
args.ktplus = True # whether to run results for KT+
args.targetkt = False # whether to run results for Target KT
args.powerkt = False # whether to run results for power KT
file_idx = range(4, 12) # since this code block runs only for Goodwin and Lotka to run results for KT+
fun_strs = ['kernel', 'x', 'x^2', 'cif'] #, 'x', 'x^2', 'cif']

In [ ]:

for filename in all_mcmc_filenames[file_idx]:
    args.filename = filename
    d, params_p, var_k =  compute_params_p(args)
    args.d = d
    params_k, params_k_power = compute_params_k(args, var_k, args.computepower, args.power)
    
    if args.ktplus:
        assert(args.power is not None)
        params_k_combo = dict()
        params_k_combo["name"] = "combo_"  + params_k["name"] + f"_{args.power}"
        params_k_combo["k"] = params_k.copy()
        params_k_combo["kpower"] = params_k_power.copy()
        params_k_combo["var"] = params_k["var"]
        params_k_combo["d"] = args.d
    
    params_k_split_list = []
    thin_str_list = []
    if args.targetkt:
        params_k_split_list.append(params_k)
        thin_str_list.append("")
    if args.powerkt:
        params_k_split_list.append(params_k_power)
        thin_str_list.append("")
    if args.ktplus:
        params_k_split_list.append(params_k_combo)
        thin_str_list.append("-plus")

    for fun_str in fun_strs:
        for params_k_split, thin_str in zip(params_k_split_list, thin_str_list):
            evaluate_fun_approx_quality(fun_str=fun_str,
                ms=range(7+1), params_p=params_p, params_k_split=params_k_split, params_k_swap=params_k, rep_ids=range(10),
                                            thin_str=thin_str, 
                             delta=0.5,
                              sample_seed=1234567, thin_seed=9876543,
                              rerun=rerun, results_dir="results_new",return_val = False)


## Hinch experiments

In [ ]:
rerun = True
args.d = int(38)
args.kernel, args.power = "imq", 0.5
args.P = "mcmc"  # target P setting; allowed values depend on the feasible arguments in compute_params_p; currently {gauss, mog, mcmc} 
args.computepower = True
args.ktplus = True # whether to run results for KT+
args.targetkt = False # whether to run results for Target KT
args.powerkt = False # whether to run results for power KT
file_idx = range(4) # since this code block runs only for Hinch
fun_strs = ['kernel', 'x', 'x^2', 'cif'] #, 'x', 'x^2', 'cif']



In [ ]:
for filename in all_mcmc_filenames[file_idx]:
    args.filename = filename
    d, params_p, var_k =  compute_params_p(args)
    args.d = d
    params_k, params_k_power = compute_params_k(args, var_k, args.computepower, args.power)
    
    if args.ktplus:
        assert(args.power is not None)
        params_k_combo = dict()
        params_k_combo["name"] = "combo_"  + params_k["name"] + f"_{args.power}"
        params_k_combo["k"] = params_k.copy()
        params_k_combo["kpower"] = params_k_power.copy()
        params_k_combo["var"] = params_k["var"]
        params_k_combo["d"] = args.d

    params_k_split_list = []
    thin_str_list = []
    if args.targetkt:
        params_k_split_list.append(params_k)
        thin_str_list.append("")
    if args.powerkt:
        params_k_split_list.append(params_k_power)
        thin_str_list.append("")
    if args.ktplus:
        params_k_split_list.append(params_k_combo)
        thin_str_list.append("-plus")

    for fun_str in fun_strs:
        for params_k_split, thin_str in zip(params_k_split_list, thin_str_list):
            evaluate_fun_approx_quality(fun_str=fun_str,
                ms=range(7+1), params_p=params_p, params_k_split=params_k_split, params_k_swap=params_k, rep_ids=range(10),
                                            thin_str=thin_str, 
                             delta=0.5,
                              sample_seed=1234567, thin_seed=9876543,
                              rerun=rerun, results_dir="results_new",return_val = False)


# MOG results with 4 and 8 components

In [ ]:
rerun = False # whether to rerun the integratione rror computations
args.d = 2 # d
args.kernel = "gauss" # kernel
args.P = "mog" # target P setting; allowed values depend on the feasible arguments in compute_params_p; currently {gauss, mog, mcmc} 
args.computepower = True # whether to compute results for power KT (same as root KT when power = 0.5)
args.power = 0.5 # power of the kernel to be used for power KT and KT+
args.ktplus = True # whether to run results for KT+
fun_strs = ['kernel'] # which functions to evaluate

In [ ]:
# for mog P the only free parameter is number of components M; everything else is computed in compute_params_p/compute_params_k
Ms = [4, 8] 
for M in Ms:
    args.M = M
    d, params_p, var_k =  compute_params_p(args)
    params_k, params_k_power = compute_params_k(args, var_k, args.computepower, args.power)
    
    if args.ktplus: # if running KT+, need to define the KT+ kernel called as params_k_combo
        assert(args.power is not None)
        params_k_combo = dict()
        params_k_combo["name"] = "combo_"  + params_k["name"] + f"_{args.power}"
        params_k_combo["k"] = params_k.copy()
        params_k_combo["kpower"] = params_k_power.copy()
        params_k_combo["var"] = params_k["var"]
        params_k_combo["d"] = args.d
        
    for fun_str in fun_strs: # compute results for each function; 
        # if args.ktplus is trye then results for KT/powerKT/KT+ are all computed, else only for KT
        # in either case, results for ST are returned
        for params_k_split, thin_str in zip([params_k, params_k_power, params_k_combo], ["", "", "-plus"]) if args.ktplus else zip([params_k], [""]):
            fun_diff_p, fun_diff_p_sin, fun_diff_p_st, fun_diff_p_sin_st = evaluate_fun_approx_quality(fun_str=fun_str,
                ms=range(7+1), params_p=params_p, params_k_split=params_k_split, params_k_swap=params_k, rep_ids=range(10),
                                            thin_str=thin_str, 
                             delta=0.5,
                              sample_seed=1234567, thin_seed=9876543,
                              rerun=rerun, results_dir="results_new",return_val = True)
